# Data Mining Project

In [59]:
#setup
import numpy
import pandas as pd
import os
import matplotlib.pyplot as plt
import plotly
import plotly.plotly as py
from mpl_toolkits import mplot3d
plotly.tools.set_credentials_file(username='xavier2696', api_key='97cuXMXYTejnMxo9gpIk')
import re
from sklearn import preprocessing, metrics, decomposition, pipeline, dummy
import sys
import math

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
#tainan buildings dataframe
building_df = pd.read_pickle('data/building/tainan/Tainan_lat_long_removed_invalid.pkl')

In [4]:
building_df.reset_index(drop=True, inplace=True)

In [5]:
english_columns = {'_id.$oid':'id',
                   '停車空間':'parking_space',
                   '原領執照字號':'license_number',
                   '地上層數':'floors_above',
                   '地下層數':'floors_basement',
                   '地下避難面積':'underground_refuge_area',
                   '地號':'place_code',
                   '執照類別':'license_category',
                   '基地面積':'base_area',
                   '實際開工日期':'date',
                   '建築物高度':'height',
                   '建築面積':'area',
                   '建造類別':'category',
                   '戶數':'number_households',
                   '承造人':'contractor',
                   '核發執照字號':'issuing_license_number',
                   '棟數':'number_buildings',
                   '構造別':'construct',
                   '樓層概要':'floor_summary',
                   '法定空地面積':'legal_empty_space',
                   '發照日期':'date_issued',
                   '監造人':'supervisor',
                   '竣工日期':'completion_date',
                   '總樓地板面積':'total_floor_area',
                   '設計人':'designer',
                   '變更設計次數':'change_design_times',
                   '起造人代表人':'representative',
                   '門牌':'address',
                   '雜項工作物':'miscellaneous',
                  }

In [6]:
building_df.rename(index=str, columns=english_columns, inplace=True)

In [7]:
print(len(building_df))
building_df.info()

392367
<class 'pandas.core.frame.DataFrame'>
Index: 392367 entries, 0 to 392366
Data columns (total 32 columns):
id                         392367 non-null object
qtime                      392367 non-null object
parking_space              392367 non-null object
license_number             392367 non-null object
floors_above               392367 non-null object
floors_basement            392367 non-null object
underground_refuge_area    392367 non-null object
place_code                 392367 non-null object
license_category           392367 non-null object
base_area                  392367 non-null object
date                       392367 non-null object
height                     392367 non-null object
area                       392367 non-null object
category                   392367 non-null object
number_households          392367 non-null object
contractor                 392367 non-null object
issuing_license_number     392367 non-null object
number_buildings           392367 non

In [14]:
building_df[0:2]

,id,qtime,parking_space,license_number,floors_above,floors_basement,underground_refuge_area,place_code,license_category,base_area,...,supervisor,completion_date,total_floor_area,designer,change_design_times,representative,address,miscellaneous,lat,long
0,5c0ac340e0f478b5cd6935f7,,法定0輛，獎勵0輛，自設0輛,,0層,0層,0㎡,[],使用執照,0㎡,...,,年月日,0㎡,,00,,[],,,
1,5c0ac340e0f478b5cd6935f8,,法定0輛，獎勵0輛，自設0輛,,0層,0層,0㎡,[],使用執照,0㎡,...,,年月日,0㎡,,00,,[],,,


In [15]:
building_df.category.value_counts()

      239270
新建    151001
增建      2011
改建        53
修建        26
拆除         6
Name: category, dtype: int64

In [9]:
count = 0
count2 = 0
for index, row in building_df.iterrows():
    if len(row.address) == 0:
        count = count + 1
    if row.lat == '' and row.long == '':
        count2 = count2 + 1
print("Percentage of data without address:", count/len(building_df))
print("Percentage of data without lat and lon", count2/len(building_df))

Percentage of data without address: 0.2843970058644076
Percentage of data without lat and lon 0.5066888907578874


In [20]:
#row_list = []

In [21]:
#filter data of shanhua district
#for index, row in building_df.iterrows():
    #if len(row.address) > 0 and "善化" in row.address[0]['行政區']:
        #row_list.append(row)
#print("Length of shanhua district data:", len(row_list))

In [22]:
#filtered_df = pd.DataFrame(row_list)

In [23]:
#filtered_df[0:2]

In [24]:
#filtered_df.to_pickle('data/building/tainan/shanhua.pkl') 

In [25]:
#filtered_df = pd.read_pickle('data/building/tainan/shanhua.pkl')

In [26]:
#filtered_df[-10:][['id','date']]

In [27]:
#filtered_df.address[:1][0]

In [28]:
#shanhua_df = filtered_df[['id','license_number','floors_above','floors_basement','license_category','base_area',\
#                         'date', 'height', 'area', 'category', 'number_households', 'issuing_license_number',\
#                        'construct', 'floor_summary', 'date_issued', 'completion_date', 'total_floor_area',\
#                        'change_design_times', 'representative', 'address']]

In [29]:
#shanhua_df[0:2]
#a lot of data is missing in the older buildings

In [30]:
#shanhua_df[-2:]

In [12]:
def get_english_license_category(license):
    if license == '使用執照':
        return 'use'
    if license == '建造執照':
        return 'construction'
    if license == '雜項執照':
        return 'miscellaneous'
    if license == '拆除執照':
        return 'demolition'
    return 'na'

def get_english_category(category):
    if category == '新建':
        return 'new'
    if category == '增建':
        return 'additional'
    if category == '改建':
        return 'reconstruction'
    if category == '修建':
        return 'construction'
    if category == '拆除':
        return 'teardown'
    
    return 'old'

In [9]:
#columns need to complete data
#license_category, category, 
#id = []
number_floors = []
number_basements = []
license_category = []
base_area = []
date = [] #use original date because completion date has too little values
issuing_license_number = []
completion_date = []
date_issued = []
height = []
building_area = []
category = []
number_households = []
construction_type = []
total_floor_area = []
number_design_changes = []
latitude = []
longitude = []
address = []

for index,row in building_df.iterrows():
    if row.lat == '' and row.long == '':
        continue
    #number of floors
    number_floors += [int(re.sub("[^0-9]", "", row.floors_above))]
    #number of basements
    number_basements += [int(re.sub("[^0-9]", "", row.floors_basement))]
    #license_category
    license_category += [get_english_license_category(row.license_category)]
    #base area
    base_area += [float(re.sub("[^0-9.]", "", row.base_area))]
    #date
    #todo find a way to represent it as a number
    date += [row.date]
    completion_date += [row.completion_date]
    date_issued += [row.date_issued]
    issuing_license_number += [row.issuing_license_number]
    #height
    height += [float(re.sub("[^0-9.]", "", row.height))]
    #building_area
    building_area += [float(re.sub("[^0-9.]", "", row.area))]
    #category
    category += [get_english_category(row.category)]
    #number_households
    number_households += [int(re.sub("[^0-9]", "", row.number_households))]
    #construction_type
    c_type = 'unknown'
    if row.construct != '':
        c_type = row.construct
    construction_type += [c_type]
    #total floor area
    total_floor_area += [float(re.sub("[^0-9.]", "", row.total_floor_area))]
    #number design changes
    number_design_changes += [int(re.sub("[^0-9]", "", row.change_design_times))]
    #address (chinese for now, latitude and longitude later)
    address += [row.address]
    #latitude
    latitude += [float(row.lat)]
    #longitude
    longitude += [float(row.long)]

data_dictionary = {'number_floors':number_floors,
                   'number_basements':number_basements,
                   'license_category':license_category,
                   'base_area':base_area,
                   'date':date,
                   'height':height,
                   'building_area':building_area,
                   'category':category,
                   'number_households':number_households,
                   'construction_type':construction_type,
                   'total_floor_area':total_floor_area,
                   'number_design_changes':number_design_changes,
                   'latitude':latitude,
                   'longitude':longitude,
                   'address': address,
                   'completion_date':completion_date,
                   'date_issued':date_issued,
                   'issuing_license_number':issuing_license_number
                  }
buildings_english_df = pd.DataFrame(data_dictionary)
    

In [10]:
buildings_english_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193559 entries, 0 to 193558
Data columns (total 18 columns):
address                   193559 non-null object
base_area                 193559 non-null float64
building_area             193559 non-null float64
category                  193559 non-null object
completion_date           193559 non-null object
construction_type         193559 non-null object
date                      193559 non-null object
date_issued               193559 non-null object
height                    193559 non-null float64
issuing_license_number    193559 non-null object
latitude                  193559 non-null float64
license_category          193559 non-null object
longitude                 193559 non-null float64
number_basements          193559 non-null int64
number_design_changes     193559 non-null int64
number_floors             193559 non-null int64
number_households         193559 non-null int64
total_floor_area          193559 non-null float64
dtype

In [13]:
buildings_english_df.category.value_counts()

old               124734
new                67154
additional          1609
reconstruction        35
construction          21
teardown               6
Name: category, dtype: int64

In [14]:
buildings_english_df.to_pickle('data/building/tainan/buildings_english.pkl') 

In [15]:
len(buildings_english_df[buildings_english_df.construction_type == 'unknown'])

122042

In [26]:
buildings_english_df[0:2]

,address,base_area,building_area,category,construction_type,date,height,latitude,license_category,longitude,number_basements,number_design_changes,number_floors,number_households,total_floor_area
0,"[{'路街段巷弄': '150巷', '戶號': '', '村里鄰': '', '樓': '...",0.0,0.0,old,unknown,年月日,0.0,23.31041,use,120.316400,0,0,0,0,0.0
1,"[{'路街段巷弄': '生產路500巷102弄', '戶號': '1', '村里鄰': ''...",0.0,0.0,old,unknown,年月日,7.0,22.96486,use,120.229242,0,0,2,1,0.0


In [38]:
#binarization for categorical columns
#buildings_english_binarized_df = buildings_english_df.copy()

#category_binarizer = preprocessing.LabelBinarizer()
#category_binarizer.fit(buildings_english_binarized_df.category)
#buildings_english_binarized_df['category'] = category_binarizer.transform(buildings_english_binarized_df['category']).tolist()

#construction_type_binarizer = preprocessing.LabelBinarizer()
#construction_type_binarizer.fit(buildings_english_binarized_df.construction_type)
#buildings_english_binarized_df['construction_type'] = construction_type_binarizer.transform(buildings_english_binarized_df['construction_type']).tolist()

#license_category_binarizer = preprocessing.LabelBinarizer()
#license_category_binarizer.fit(buildings_english_binarized_df.license_category)
#buildings_english_binarized_df['license_category'] = license_category_binarizer.transform(buildings_english_binarized_df['license_category']).tolist()

In [39]:
#buildings_english_binarized_df[0:2]

In [16]:
#building damages
yellow_damaged_buildings_df = pd.read_csv('data/building/tainan/opendata0206earthquakedangerbuildingmarkedyellow20160311.csv')
yellow_damaged_buildings_df['damage_level'] = 1

In [17]:
len(yellow_damaged_buildings_df)

329

In [18]:
yellow_damaged_buildings_df[-2:]

,流水號,行政區,地址,經度,緯度,初勘人員,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,damage_level
327,328,關廟區,臺南市關廟區旺萊路263號,120.337668,22.955858,楊王進、黃柏鈞,NaN,NaN,NaN,NaN,1
328,329,鹽水區,臺南市鹽水區孫厝里1鄰孫厝寮4-3號,120.226009,23.289025,周志安,NaN,NaN,NaN,NaN,1


In [19]:
red_damaged_buildings_df = pd.read_csv('data/building/tainan/opendata0206earthquakedangerbuildingmarkedred20160311.csv')
red_damaged_buildings_df['damage_level'] = 2

In [20]:
len(red_damaged_buildings_df)

250

In [21]:
red_damaged_buildings_df[-2:]

,流水號,行政區,地址,經度,緯度,初勘人員,damage_level
248,249,鹽水區,臺南市鹽水區康樂路2號,120.267411,23.317453,蔡炅霖、周志安,2
249,250,鹽水區,臺南市鹽水區南港里9鄰?頭港198-15號,120.249381,23.254387,周志安,2


In [22]:
english_columns2 = {'經度':'longitude',
                   '緯度':'latitude',
                  }
combined_df = pd.concat([red_damaged_buildings_df, yellow_damaged_buildings_df], axis=0, sort=False)
damage_df = combined_df.loc[:,('經度','緯度','damage_level')]
damage_df.rename(index=str, columns=english_columns2, inplace=True)
print(len(damage_df))
damage_df[:2]

579


,longitude,latitude,damage_level
0,120.230098,22.962252,2
1,120.234832,22.970249,2


In [23]:
damage_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 579 entries, 0 to 328
Data columns (total 3 columns):
longitude       578 non-null float64
latitude        578 non-null float64
damage_level    579 non-null int64
dtypes: float64(2), int64(1)
memory usage: 18.1+ KB


In [24]:
damage_df.to_pickle('data/building/tainan/buildings_damaged.pkl')

In [25]:
#liquefaction
liquefaction_df = pd.read_pickle('data/geographical/liquefaction.pkl')

In [26]:
liquefaction_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 446 entries, 0 to 445
Data columns (total 5 columns):
coordinates    446 non-null object
type           446 non-null object
area           446 non-null object
classify       446 non-null int64
level          446 non-null object
dtypes: int64(1), object(4)
memory usage: 17.5+ KB


In [27]:
liquefaction_df[0:4]

,coordinates,type,area,classify,level
0,"[[[120.27871193, 22.89375869], [120.27871671, ...",Polygon,TaiNan,1,中潛勢
1,"[[[120.27871193, 22.89375869], [120.27870716, ...",Polygon,TaiNan,1,中潛勢
2,"[[[120.27185562, 22.90004867], [120.27186044, ...",Polygon,TaiNan,1,中潛勢
3,"[[[120.28842552, 22.90012372], [120.28745082, ...",Polygon,TaiNan,1,中潛勢


In [28]:
liquefaction_df.classify.value_counts()

1    250
2    114
0     82
Name: classify, dtype: int64

In [29]:
liquefaction_df.coordinates[0][0][2]

[120.27774206, 22.89285125]

In [40]:
def get_liquefaction(lon, lat, df):
    for index, row in df.iterrows():
        if len(row.coordinates) > 0 and len(row.coordinates[0]) > 0:
            for coordinate in row.coordinates[0]:
                if "{0:.4f}".format(lon) == "{0:.4f}".format(coordinate[0]) and "{0:.4f}".format(lat) == "{0:.4f}".format(coordinate[1]):
                    return row.classify
            for coordinate in row.coordinates[0]:
                if "{0:.3f}".format(lon) == "{0:.3f}".format(coordinate[0]) and "{0:.3f}".format(lat) == "{0:.3f}".format(coordinate[1]):
                    return row.classify
            for coordinate in row.coordinates[0]:
                if "{0:.2f}".format(lon) == "{0:.2f}".format(coordinate[0]) and "{0:.2f}".format(lat) == "{0:.2f}".format(coordinate[1]):
                    return row.classify
            for coordinate in row.coordinates[0]:
                if "{0:.1f}".format(lon) == "{0:.1f}".format(coordinate[0]) and "{0:.1f}".format(lat) == "{0:.1f}".format(coordinate[1]):
                    return row.classify
    return 0
                
def get_damage_level(lon, lat, df):
    for index, row in df.iterrows():
        if "{0:.6f}".format(row.latitude) == "{0:.6f}".format(lat) and "{0:.6f}".format(row.longitude) == "{0:.6f}".format(lon):
            #print('match')
            return row.damage_level
    for index, row in df.iterrows():
        if "{0:.5f}".format(row.latitude) == "{0:.5f}".format(lat) and "{0:.5f}".format(row.longitude) == "{0:.5f}".format(lon):
            #print('match')
            return row.damage_level
    for index, row in df.iterrows():
        if "{0:.4f}".format(row.latitude) == "{0:.4f}".format(lat) and "{0:.4f}".format(row.longitude) == "{0:.4f}".format(lon):
            #print('match')
            return row.damage_level
    for index, row in df.iterrows():
        if "{0:.3f}".format(row.latitude) == "{0:.3f}".format(lat) and "{0:.3f}".format(row.longitude) == "{0:.3f}".format(lon):
            #print('match')
            return row.damage_level
    return 0

In [31]:
print(get_liquefaction(120.27774206,22.89285125,liquefaction_df))

1


In [32]:
print(buildings_english_df.latitude[0:10])

0    23.31041
1    22.96486
2    22.96486
3    22.96486
4    22.96486
5    22.96486
6    22.96486
7    22.96486
8    22.96486
9    22.96486
Name: latitude, dtype: float64


In [33]:
print(damage_df.latitude[0:10])

0    22.962252
1    22.970249
2    23.003243
3    23.003243
4    23.003243
5    23.003243
6    23.003169
7    23.003169
8    23.003169
9    23.003169
Name: latitude, dtype: float64


In [41]:
damage_levels = []
for index, row in buildings_english_df.iterrows():
    damage_levels += [int(get_damage_level(row.longitude, row.latitude, damage_df))]

In [42]:
buildings_english_df['damage_level'] = damage_levels

In [43]:
buildings_english_df.damage_level.value_counts()

0    191799
2      1214
1       546
Name: damage_level, dtype: int64

In [44]:
buildings_english_df.to_pickle('data/building/tainan/buildings_with_damages.pkl')

In [45]:
buildings_english_df = pd.read_pickle('data/building/tainan/buildings_with_damages.pkl')

In [ ]:

#merged_df = buildings_english_df.merge(damage_df, how='left', on=['latitude', 'longitude'])

In [46]:
buildings_english_df.damage_level[0:10]

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
Name: damage_level, dtype: int64

In [47]:
liquefactions = []
for index, row in buildings_english_df.iterrows():
    liquefactions += [int(get_liquefaction(row.longitude, row.latitude, liquefaction_df))]

In [48]:
buildings_english_df['liquefaction'] = liquefactions
buildings_english_df.liquefaction.value_counts()

1    173155
0     20404
Name: liquefaction, dtype: int64

In [49]:
buildings_english_df.to_pickle('data/building/tainan/buildings_with_everything.pkl')

In [ ]:
buildings_english_df[buildings_english_df['liquefaction'] == -1][-10:]

In [ ]:
buildings_english_df[buildings_english_df['liquefaction'] != -1][0:10]

In [50]:
buildings_df = pd.read_pickle('data/building/tainan/buildings_with_everything.pkl')

In [51]:
buildings_df[buildings_df.damage_level != 0][0:10]

,address,base_area,building_area,category,completion_date,construction_type,date,date_issued,height,issuing_license_number,latitude,license_category,longitude,number_basements,number_design_changes,number_floors,number_households,total_floor_area,damage_level,liquefaction
164,"[{'村里鄰': '', '戶號': '1', '樓': '', '號': '15號', '...",0.0,0.0,old,年月日,unknown,年月日,067年01月24日,12.00,(067)南工使字第40728號,23.005583,use,120.269391,0,0,3,1,0.0,1,1
470,"[{'村里鄰': '', '戶號': '1', '樓': '', '號': '9號', '行...",0.0,0.0,old,年月日,unknown,年月日,067年03月06日,9.10,(067)南工使字第41033號,22.964366,use,120.297488,0,0,3,1,0.0,2,1
471,"[{'村里鄰': '', '戶號': '1', '樓': '', '號': '17號', '...",0.0,0.0,old,年月日,unknown,年月日,067年03月06日,12.10,(067)南工使字第41034號,22.964366,use,120.297488,0,0,4,1,0.0,2,1
1060,"[{'村里鄰': '', '戶號': '1', '樓': '', '號': '21號', '...",0.0,0.0,old,年月日,unknown,年月日,067年05月18日,12.00,(067)南工字第41639號,23.317367,use,120.266649,0,0,4,1,0.0,2,0
1146,"[{'村里鄰': '', '戶號': '1', '樓': '', '號': '38號', '...",0.0,0.0,old,年月日,unknown,年月日,067年05月31日,0.00,(067)南工字第41728號,23.317367,use,120.266649,0,0,2,1,0.0,2,0
3266,"[{'村里鄰': '', '戶號': '1', '樓': '', '號': '28號', '...",0.0,0.0,old,年月日,unknown,年月日,067年11月30日,12.20,(067)南工字第43965號,23.317367,use,120.266649,0,0,4,1,0.0,2,0
3374,"[{'村里鄰': '', '戶號': '1', '樓': '', '號': '30號', '...",0.0,0.0,old,年月日,unknown,年月日,067年12月11日,12.20,(067)南工字第44091號,23.317367,use,120.266649,0,0,4,1,0.0,2,0
3499,"[{'村里鄰': '', '戶號': '1', '樓': '', '號': '10號', '...",0.0,0.0,old,年月日,unknown,年月日,067年12月13日,12.20,(067)南工字第44216號,22.996904,use,120.198159,0,0,4,1,0.0,2,1
4547,"[{'村里鄰': '', '戶號': '1', '樓': '', '號': '210號', ...",0.0,0.0,old,年月日,unknown,年月日,068年01月23日,4.45,(068)南工字第45401號,23.123718,use,120.468816,0,0,1,1,0.0,2,0
6858,"[{'村里鄰': '', '戶號': '1', '樓': '', '號': '34號', '...",0.0,0.0,old,年月日,unknown,年月日,068年05月23日,24.60,(068)南工字第47722號,23.317367,use,120.266649,0,0,7,1,0.0,2,0


In [52]:
earthquakes_df = pd.read_pickle('data/earthquake/earthquakes_Tainan_withData_withPeakValue_dropUseless_removeDuplicate.pkl')

In [53]:
def get_earthquake_data(lat, lon, df):
    min_difference = sys.float_info.max
    earthquake_data = [0,0,0,0,0,0,0,0,0,0,0]
    for index, row in df.iterrows():
        difference = math.sqrt(math.pow(abs(row.station_latitude - lat), 2) + math.pow(abs(row.station_longitude - lon), 2))
        if difference < min_difference:
            min_difference = difference
            earthquake_data = [row.e_PGA, row.e_PGV, row.e_PGD, row.n_PGA, row.n_PGV, row.n_PGD, row.u_PGA, row.u_PGV, row.u_PGD, row.station_latitude, row.station_longitude]
    return earthquake_data

In [56]:
buildings_df.liquefaction.value_counts()

1    173155
0     20404
Name: liquefaction, dtype: int64

In [60]:
#add closest station data to buildings_df
e_PGA = []
e_PGV = []
e_PGD = []
n_PGA = []
n_PGV = []
n_PGD = []
u_PGA = []
u_PGV = []
u_PGD = []
for index, row in buildings_df.iterrows():
    e_data = get_earthquake_data(row.latitude, row.longitude, earthquakes_df)
    e_PGA += [e_data[0]]
    e_PGV += [e_data[1]]
    e_PGD += [e_data[2]]
    n_PGA += [e_data[3]]
    n_PGV += [e_data[4]]
    n_PGD += [e_data[5]]
    u_PGA += [e_data[6]]
    u_PGV += [e_data[7]]
    u_PGD += [e_data[8]]
    
buildings_df['e_PGA'] = e_PGA
buildings_df['e_PGV'] = e_PGV
buildings_df['e_PGD'] = e_PGD
buildings_df['n_PGA'] = n_PGA
buildings_df['n_PGV'] = n_PGV
buildings_df['n_PGD'] = n_PGD
buildings_df['u_PGA'] = u_PGA
buildings_df['u_PGV'] = u_PGV
buildings_df['u_PGD'] = u_PGD
    

In [61]:
buildings_df.to_pickle('all_tainan_data.pkl')

In [62]:
buildings_df.category.value_counts()

old               124734
new                67154
additional          1609
reconstruction        35
construction          21
teardown               6
Name: category, dtype: int64